In [1]:
import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

randomlist = []
df_demand = {}
random.seed(16) # Use the same random seed
for i in range(0,20):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(20):
    df_demand["D"+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v

df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0

np.random.seed(16) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,40, size=(sum,20))).astype(float)

random.seed(16) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(16) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(16) # Use the same random seed
doc = {}
for i in range(0,20):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom
df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]
dk = df_tdkom.transpose()
costs_t = dk.to_dict()
cols = sorted(df_demand.keys())

In [2]:
# print(df_supply['S1'])
# break

costs1=copy.deepcopy(costs_t)
# costs2=copy.deepcopy(costs_t)
# costs3=copy.deepcopy(costs_t)
# for i in df_supply:
#     mi=min(costs_t[i].values())
#     # print(costs_t[i])
#     # print(mi)
#     for j in costs2[i]:
#         costs2[i][j]-=mi
# # print(costs2)
# for i in df_demand :
#     mi=10000
#     for j in df_supply:
#         if costs_t[j][i]<mi :
#             mi=costs_t[j][i]
#     for j in df_supply:
#         costs3[j][i]=costs3[j][i]-mi 
# # print(costs3)

# for i in df_demand:
#     for j in df_supply:
#         costs_t[j][i]= costs2[j][i]+costs3[j][i]



res = dict((k, defaultdict(int)) for k in costs_t)
g = {}
for x in df_supply:
    g[x] = sorted(costs_t[x].keys(), key=lambda g: costs_t[x][g])
for x in df_demand:
    g[x] = sorted(costs_t.keys(), key=lambda g: costs_t[g][x])
# print(costs1)
flag=0
while df_supply and df_demand:

    d = {}
    # print(df_demand,df_supply)

    s = {}
    for x in df_supply:
        s[x] = 0
        for y in df_demand:
            s[x]=s[x] + costs_t[x][y]- costs_t[x][g[x][0]]

    mp = max(list(s.values()))
    mi=1000000
    l=[]
    for x in df_supply :
        if s[x]==mp:
            l.append(x)
            if costs_t[x][g[x][0]]<mi:
                mi=costs_t[x][g[x][0]]
    sup="S"
    ma=0
    # print(demand)
    for x in l :
        # print(g[x][0])
        if costs_t[x][g[x][0]]==mi and df_demand[g[x][0]]>=ma:
            sup=x 
    # print(costs[sup][g[sup][0]], len(df_supply))
    if costs_t[sup][g[sup][0]]!=0 or len(df_supply)==1:
        # print("here")
        v=min(df_supply[sup],df_demand[g[sup][0]])
        # print(sup,demand,v)
        df_demand[g[sup][0]]-=v
        df_supply[sup]-=v 
        res[sup][g[sup][0]]+=v 
        dem = g[sup][0]
        # print(sup,dem,v)
        if df_demand[dem]==0:
            for k, n in df_supply.items():
                if n != 0:
                    g[k].remove(dem)
            del g[dem]
            del df_demand[dem]
        if df_supply[sup]==0:
            for k, n in df_demand.items():
                if n != 0:
                    g[k].remove(sup)
            del g[sup]
            del df_supply[sup]

    else :
        mind="S"
        ma=-1
        # print(sup)
        for x in df_supply :
            if s[x]>ma and x!=sup :
                ma=s[x]
                mind=x
        gv1=0
        # print(mind,"this")
        for y in df_demand:
            if costs_t[sup][y] > costs_t[mind][y]:
                gv1+=1 
            else:
                gv1-=1
        if gv1>=0 :
            v=min(df_supply[sup],df_demand[g[sup][0]])
            # print(sup,demand,v)
            df_demand[g[sup][0]]-=v
            df_supply[sup]-=v 
            res[sup][g[sup][0]]+=v 
            dem=g[sup][0]
            # print(sup,dem,v)
            if df_demand[dem]==0:
                for k, n in df_supply.items():
                    if n != 0:
                        g[k].remove(dem)
                del g[dem]
                del df_demand[dem]
            if df_supply[sup]==0:
                for k, n in df_demand.items():
                    if n != 0:
                        g[k].remove(sup)
                del g[sup]
                del df_supply[sup]
        else :
            sup=mind 
            v=min(df_supply[sup],df_demand[g[sup][0]])

            df_demand[g[sup][0]]-=v
            df_supply[sup]-=v 
            res[sup][g[sup][0]]+=v 
            dem=g[sup][0]
            # print(sup,dem,v)
            if df_demand[dem]==0:
                for k, n in df_supply.items():
                    if n != 0:
                        g[k].remove(dem)
                del g[dem]
                del df_demand[dem]
            if df_supply[sup]==0:
                for k, n in df_demand.items():
                    if n != 0:
                        g[k].remove(sup)
                del g[sup]
                del df_supply[sup]
    # break


print(df_demand,df_supply)

# print(costs1)
cost = 0
for g in sorted(costs1):
    # print (g, " ",)
    for n in cols:
        y = res[g][n]
        if y != 0:
            pass
            # print (y,)
        cost += y * costs1[g][n]
        # print ("  ",)
    # print(" ")
# print ("Total Cost = ", cost)



{} {}


In [3]:
cost

279162.37855578243

stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time))